# F1 Strategy Regret — Data Exploration

Initial scratchpad for inspecting the raw CSV exports powering the strategy regret pipeline. Use this notebook to build intuition about the available fields and to prototype transformations before promoting them into the production data processing jobs.

## Setup


In [1]:
from pathlib import Path

import pandas as pd

pd.set_option('display.max_columns', 50)

PROJECT_ROOT = Path.cwd().resolve().parents[0]
DATA_DIR = PROJECT_ROOT / 'data'
DATA_DIR


PosixPath('/Users/maccarlton/FunProjects/f1/data')

## Helpers


In [2]:
def load_table(name: str, *, parse_dates=None, **read_csv_kwargs) -> pd.DataFrame:
    """Load a CSV from the shared data directory with a quick shape summary."""

    path = DATA_DIR / name
    df = pd.read_csv(path, parse_dates=parse_dates, **read_csv_kwargs)
    print(f"{name}: {df.shape[0]:,} rows x {df.shape[1]} cols")
    return df


## Core Tables Overview


In [3]:
races = load_table('races.csv', parse_dates=['date']).rename(columns={'year': 'season', 'name': 'race_name'})
results = load_table('results.csv')
pit_stops = load_table('pit_stops.csv')
lap_times = load_table('lap_times.csv')

races.head()


races.csv: 1,125 rows x 18 cols
results.csv: 26,759 rows x 18 cols
pit_stops.csv: 11,371 rows x 7 cols
lap_times.csv: 589,081 rows x 6 cols


,raceId,season,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


## Focus Seasons 2020–2024


In [8]:
TARGET_SEASONS = list(range(2020, 2025))

races = races.loc[races['season'].isin(TARGET_SEASONS)].reset_index(drop=True)
race_ids = set(races['raceId'])

results = results[results['raceId'].isin(race_ids)].reset_index(drop=True)
pit_stops = pit_stops[pit_stops['raceId'].isin(race_ids)].reset_index(drop=True)
lap_times = lap_times[lap_times['raceId'].isin(race_ids)].reset_index(drop=True)

(
    races.groupby('season')['raceId']
    .nunique()
    .rename('num_races')
)


season
2020    17
2021    22
2022    22
2023    22
2024    24
Name: num_races, dtype: int64

In [9]:
(
    races
    .groupby('season')
    .agg(num_races=('raceId', 'nunique'))
    .assign(first_round=races.groupby('season')['round'].min())
)


,num_races,first_round
season,,
2020,17,1
2021,22,1
2022,22,1
2023,22,1
2024,24,1


## Reference Tables


In [10]:
drivers = load_table('drivers.csv')
constructors = load_table('constructors.csv')
circuits = load_table('circuits.csv')
status = load_table('status.csv')

drivers.head(3)


drivers.csv: 861 rows x 9 cols
constructors.csv: 212 rows x 5 cols
circuits.csv: 77 rows x 9 cols
status.csv: 139 rows x 2 cols


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg


## Race Results with Labels


In [18]:
drivers = drivers.assign(driver_name=drivers['forename'] + ' ' + drivers['surname'])
constructors = constructors.rename(columns={'name': 'constructor_name', 'nationality': 'constructor_nationality'})
status = status.rename(columns={'status': 'status_label'})

race_meta_cols = ['raceId', 'season', 'round', 'date']
race_name_col = 'race_name' if 'race_name' in races.columns else 'name'
if race_name_col not in race_meta_cols:
    race_meta_cols.append(race_name_col)

races_meta = races[race_meta_cols].copy()
if 'name' in races_meta.columns and 'race_name' not in races_meta.columns:
    races_meta = races_meta.rename(columns={'name': 'race_name'})

race_results = (
    results
    .merge(races_meta, on='raceId', how='left')
    .merge(drivers[['driverId', 'code', 'driver_name', 'nationality']], on='driverId', how='left')
    .merge(
        constructors[['constructorId', 'constructor_name', 'constructor_nationality']],
        on='constructorId',
        how='left',
    )
    .merge(status[['statusId', 'status_label']], on='statusId', how='left')
)

race_results[['season', 'race_name', 'round', 'code', 'constructor_name', 'positionText', 'points', 'status_label']].head()


,season,race_name,round,code,constructor_name,positionText,points,status_label
0,2020,Austrian Grand Prix,1,BOT,Mercedes,1,25.0,Finished
1,2020,Austrian Grand Prix,1,LEC,Ferrari,2,18.0,Finished
2,2020,Austrian Grand Prix,1,NOR,McLaren,3,16.0,Finished
3,2020,Austrian Grand Prix,1,HAM,Mercedes,4,12.0,Finished
4,2020,Austrian Grand Prix,1,SAI,McLaren,5,10.0,Finished


### Pit Stop Cadence

Quick check on average pit stops per driver per race to gauge the distribution we need to capture inside the simulation.


In [19]:
pit_stops_with_meta = pit_stops.merge(races[['raceId', 'season']], on='raceId', how='left')
pit_stop_distribution = (
    pit_stops_with_meta
    .groupby(['season', 'raceId', 'driverId'])
    .size()
    .rename('stop_count')
    .reset_index()
)

(
    pit_stop_distribution
    .groupby(['season', 'stop_count'])
    .size()
    .rename('num_driver_races')
    .reset_index()
)


,season,stop_count,num_driver_races
0,2020,1,125
1,2020,2,127
2,2020,3,44
3,2020,4,4
4,2020,5,11
5,2020,6,2
6,2021,1,149
7,2021,2,144
8,2021,3,66
9,2021,4,28


### Lap Time Summary

Baseline lap pace spread, which is a helpful sanity check before fitting stint degradation models.


In [20]:
lap_times['milliseconds'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9])


count    1.165770e+05
mean     9.679079e+04
std      9.448751e+04
min      5.540400e+04
10%      7.537400e+04
25%      8.135900e+04
50%      8.930100e+04
75%      9.971200e+04
90%      1.103920e+05
max      3.204155e+06
Name: milliseconds, dtype: float64

In [21]:
lap_times_with_meta = lap_times.merge(races[['raceId', 'season']], on='raceId', how='left')
lap_time_quantiles = (
    lap_times_with_meta
    .groupby('season')['milliseconds']
    .quantile([0.1, 0.5, 0.9])
    .unstack(level=-1)
    .rename(columns={0.1: 'p10_ms', 0.5: 'p50_ms', 0.9: 'p90_ms'})
)
lap_time_quantiles.round(1)


,p10_ms,p50_ms,p90_ms
season,,,
2020,68848.8,86684.0,111084.2
2021,72527.7,87373.0,106593.6
2022,76787.0,91094.0,116297.6
2023,76515.0,90427.5,109729.5
2024,77235.3,89290.5,108461.0


## Regret Proxy (Race Pace Delta)


In [22]:
regret_base = race_results.copy()
regret_base['milliseconds'] = pd.to_numeric(regret_base['milliseconds'], errors='coerce')
regret_base['laps'] = pd.to_numeric(regret_base['laps'], errors='coerce')

finish_mask = (
    regret_base['status_label'].str.contains('Finished', case=False, na=False)
    | regret_base['status_label'].str.startswith('+', na=False)
)

regret_finishers = regret_base.loc[finish_mask].dropna(subset=['milliseconds', 'laps']).copy()

regret_finishers['race_best_ms'] = regret_finishers.groupby('raceId')['milliseconds'].transform('min')
regret_finishers['best_same_lap_ms'] = (
    regret_finishers.groupby(['raceId', 'laps'])['milliseconds'].transform('min')
)
regret_finishers['regret_vs_winner_ms'] = regret_finishers['milliseconds'] - regret_finishers['race_best_ms']
regret_finishers['regret_same_lap_ms'] = regret_finishers['milliseconds'] - regret_finishers['best_same_lap_ms']
regret_finishers['max_laps_in_race'] = regret_finishers.groupby('raceId')['laps'].transform('max')
regret_finishers['on_lead_lap'] = regret_finishers['laps'] == regret_finishers['max_laps_in_race']

constructor_regret = (
    regret_finishers.loc[regret_finishers['on_lead_lap']]
    .groupby(['season', 'constructor_name'])['regret_vs_winner_ms']
    .median()
    .reset_index()
    .assign(avg_regret_s=lambda df: df['regret_vs_winner_ms'] / 1000)
    .drop(columns='regret_vs_winner_ms')
)

constructor_regret.sort_values(['season', 'avg_regret_s']).groupby('season').head(5)


,season,constructor_name,avg_regret_s
5,2020,Mercedes,6.4455
8,2020,Renault,19.3680
4,2020,McLaren,21.5180
7,2020,Red Bull,24.1770
1,2020,AlphaTauri,26.9770
17,2021,Mercedes,14.0970
18,2021,Red Bull,16.7965
15,2021,Haas F1 Team,29.5070
10,2021,Alfa Romeo,32.9115
19,2021,Williams,43.3790


In [24]:
lead_lap_highlights = (
    regret_finishers.loc[regret_finishers['on_lead_lap']]
    .sort_values('regret_vs_winner_ms', ascending=False)
    [['season', 'race_name', 'driver_name', 'constructor_name', 'positionText', 'regret_vs_winner_ms', 'regret_same_lap_ms']]
    .assign(
        regret_vs_winner_s=lambda df: df['regret_vs_winner_ms'] / 1000,
        regret_same_lap_s=lambda df: df['regret_same_lap_ms'] / 1000,
    )
    .head(10)
)
lead_lap_highlights


,season,race_name,driver_name,constructor_name,positionText,regret_vs_winner_ms,regret_same_lap_ms,regret_vs_winner_s,regret_same_lap_s
1992,2024,Azerbaijan Grand Prix,Guanyu Zhou,Sauber,14,148841.0,148841.0,148.841,148.841
1991,2024,Azerbaijan Grand Prix,Daniel Ricciardo,RB F1 Team,13,146907.0,146907.0,146.907,146.907
1990,2024,Azerbaijan Grand Prix,Pierre Gasly,Alpine F1 Team,12,117189.0,117189.0,117.189,117.189
1691,2024,Saudi Arabian Grand Prix,Kevin Magnussen,Haas F1 Team,12,112373.0,112373.0,112.373,112.373
1457,2023,Belgian Grand Prix,Nico Hülkenberg,Haas F1 Team,18,110450.0,110450.0,110.450,110.450
929,2022,Azerbaijan Grand Prix,Esteban Ocon,Alpine F1 Team,10,108184.0,108184.0,108.184,108.184
1137,2022,Japanese Grand Prix,Pierre Gasly,AlphaTauri,18,108091.0,108091.0,108.091,108.091
1054,2022,Belgian Grand Prix,Daniel Ricciardo,McLaren,15,107163.0,107163.0,107.163,107.163
1053,2022,Belgian Grand Prix,Guanyu Zhou,Alfa Romeo,14,106252.0,106252.0,106.252,106.252
630,2021,Russian Grand Prix,Lance Stroll,Aston Martin,11,106198.0,106198.0,106.198,106.198


## Unique Pit Strategy Outcomes

Approximate pit strategy uniqueness by comparing each driver's pit-stop lap sequence within a race (compound data is not bundled in these CSVs). Focus on cases where one-of-one strategies produced podiums or wins.


In [25]:
strategy_features = (
    pit_stops_with_meta
    .sort_values(['raceId', 'driverId', 'stop'])
    .groupby(['raceId', 'driverId'])
    .agg(
        stop_count=('stop', 'count'),
        stop_laps_tuple=('lap', lambda laps: tuple(int(x) for x in laps)),
    )
    .reset_index()
)
strategy_features['stop_laps_repr'] = strategy_features['stop_laps_tuple'].apply(
    lambda laps: ', '.join(map(str, laps)) if laps else 'no stops'
)
strategy_features['strategy_key'] = strategy_features.apply(
    lambda row: f"{row['stop_count']}|{row['stop_laps_repr']}",
    axis=1,
)

strategy_all = (
    race_results[
        ['raceId', 'driverId', 'driver_name', 'constructor_name', 'season', 'race_name', 'positionOrder', 'positionText']
    ]
    .merge(
        strategy_features[['raceId', 'driverId', 'stop_count', 'stop_laps_tuple', 'stop_laps_repr', 'strategy_key']],
        on=['raceId', 'driverId'],
        how='left',
    )
)

strategy_all['stop_count'] = strategy_all['stop_count'].fillna(0).astype(int)
strategy_all['stop_laps_tuple'] = strategy_all['stop_laps_tuple'].apply(
    lambda laps: laps if isinstance(laps, tuple) else tuple()
)
strategy_all['stop_laps_repr'] = strategy_all['stop_laps_tuple'].apply(
    lambda laps: ', '.join(map(str, laps)) if laps else 'no stops'
)
strategy_all['strategy_key'] = strategy_all.apply(
    lambda row: f"{row['stop_count']}|{row['stop_laps_repr']}",
    axis=1
)
strategy_all['positionOrder'] = pd.to_numeric(strategy_all['positionOrder'], errors='coerce')

strategy_occurrences = (
    strategy_all.groupby(['raceId', 'strategy_key'])
    .size()
    .reset_index(name='strategy_occurrences')
)
strategy_all = strategy_all.merge(strategy_occurrences, on=['raceId', 'strategy_key'], how='left')
strategy_all['strategy_unique'] = strategy_all['strategy_occurrences'] == 1

strategy_all.head()


,raceId,driverId,driver_name,constructor_name,season,race_name,positionOrder,positionText,stop_count,stop_laps_tuple,stop_laps_repr,strategy_key,strategy_occurrences,strategy_unique
0,1031,822,Valtteri Bottas,Mercedes,2020,Austrian Grand Prix,1,1,2,"(26, 56)","26, 56","2|26, 56",5,False
1,1031,844,Charles Leclerc,Ferrari,2020,Austrian Grand Prix,2,2,3,"(26, 51, 56)","26, 51, 56","3|26, 51, 56",6,False
2,1031,846,Lando Norris,McLaren,2020,Austrian Grand Prix,3,3,3,"(26, 51, 56)","26, 51, 56","3|26, 51, 56",6,False
3,1031,1,Lewis Hamilton,Mercedes,2020,Austrian Grand Prix,4,4,2,"(26, 56)","26, 56","2|26, 56",5,False
4,1031,832,Carlos Sainz,McLaren,2020,Austrian Grand Prix,5,5,3,"(26, 52, 56)","26, 52, 56","3|26, 52, 56",1,True


In [26]:
podium_strategies = strategy_all[strategy_all['positionOrder'] <= 3].copy()
unique_podiums = (
    podium_strategies[podium_strategies['strategy_unique']]
    .sort_values(['season', 'raceId', 'positionOrder'])
    [['season', 'race_name', 'driver_name', 'constructor_name', 'positionText', 'stop_count', 'stop_laps_repr']]
)
unique_podiums.head(10)


,season,race_name,driver_name,constructor_name,positionText,stop_count,stop_laps_repr
20,2020,Styrian Grand Prix,Lewis Hamilton,Mercedes,1,1,27
22,2020,Styrian Grand Prix,Max Verstappen,Red Bull,3,2,"24, 68"
40,2020,Hungarian Grand Prix,Lewis Hamilton,Mercedes,1,3,"3, 37, 66"
41,2020,Hungarian Grand Prix,Max Verstappen,Red Bull,2,2,"4, 36"
42,2020,Hungarian Grand Prix,Valtteri Bottas,Mercedes,3,3,"2, 33, 49"
80,2020,70th Anniversary Grand Prix,Max Verstappen,Red Bull,1,2,"26, 32"
81,2020,70th Anniversary Grand Prix,Lewis Hamilton,Mercedes,2,2,"14, 41"
82,2020,70th Anniversary Grand Prix,Valtteri Bottas,Mercedes,3,2,"13, 32"
100,2020,Spanish Grand Prix,Lewis Hamilton,Mercedes,1,2,"23, 50"
101,2020,Spanish Grand Prix,Max Verstappen,Red Bull,2,2,"21, 41"


In [27]:
podium_summary = (
    podium_strategies
    .groupby('season')
    .agg(
        total_podiums=('driverId', 'size'),
        unique_strategy_podiums=('strategy_unique', 'sum'),
    )
    .assign(unique_share=lambda df: (df['unique_strategy_podiums'] / df['total_podiums']).round(3))
)
podium_summary['unique_strategy_podiums'] = podium_summary['unique_strategy_podiums'].astype(int)
podium_summary


,total_podiums,unique_strategy_podiums,unique_share
season,,,
2020,51,33,0.647
2021,66,44,0.667
2022,66,46,0.697
2023,66,44,0.667
2024,72,49,0.681


In [28]:
unique_wins = (
    podium_strategies[
        (podium_strategies['strategy_unique'])
        & (podium_strategies['positionOrder'] == 1)
    ][['season', 'race_name', 'driver_name', 'constructor_name', 'stop_count', 'stop_laps_repr']]
)
unique_wins


,season,race_name,driver_name,constructor_name,stop_count,stop_laps_repr
20,2020,Styrian Grand Prix,Lewis Hamilton,Mercedes,1,27
40,2020,Hungarian Grand Prix,Lewis Hamilton,Mercedes,3,"3, 37, 66"
80,2020,70th Anniversary Grand Prix,Max Verstappen,Red Bull,2,"26, 32"
100,2020,Spanish Grand Prix,Lewis Hamilton,Mercedes,2,"23, 50"
140,2020,Italian Grand Prix,Pierre Gasly,AlphaTauri,2,"19, 26"
...,...,...,...,...,...,...
1919,2024,Belgian Grand Prix,Lewis Hamilton,Mercedes,2,"11, 26"
1939,2024,Dutch Grand Prix,Lando Norris,McLaren,1,28
1999,2024,Singapore Grand Prix,Lando Norris,McLaren,1,30
2039,2024,Mexico City Grand Prix,Carlos Sainz,Ferrari,1,32


## Race Feature Engineering (2022–2024)

Assemble race-level features for recent seasons to power content summaries and model-friendly signals.


In [31]:
from functools import reduce
from urllib.parse import quote_plus

target_summary_seasons = list(range(2022, 2025))
race_scope = race_results[race_results['season'].isin(target_summary_seasons)].copy()

print(f"Analyzing {race_scope['raceId'].nunique()} races across {target_summary_seasons}")
print(race_scope[['season', 'race_name']].drop_duplicates().head())

analysis_pit = pit_stops_with_meta[pit_stops_with_meta['season'].isin(target_summary_seasons)].copy()
strategy_scope = strategy_all[strategy_all['season'].isin(target_summary_seasons)].copy()
regret_scope = regret_finishers[regret_finishers['season'].isin(target_summary_seasons)].copy()

print(f"Pit stop rows in scope: {len(analysis_pit):,}")
print(f"Strategy rows in scope: {len(strategy_scope):,}")
print(f"Regret rows in scope: {len(regret_scope):,}")


Analyzing 68 races across [2022, 2023, 2024]
     season                  race_name
780    2022         Bahrain Grand Prix
800    2022   Saudi Arabian Grand Prix
820    2022      Australian Grand Prix
840    2022  Emilia Romagna Grand Prix
860    2022           Miami Grand Prix
Pit stop rows in scope: 2,542
Strategy rows in scope: 1,359
Regret rows in scope: 870


In [33]:
from functools import reduce

_required = ['race_scope', 'analysis_pit', 'strategy_scope', 'regret_scope']
_missing = [name for name in _required if name not in globals()]
if _missing:
    raise NameError(f"Missing prerequisite data frames: {_missing}. Rerun the setup cell above.")

podium_df = (
    race_scope[race_scope['positionOrder'] <= 3]
    .sort_values(['raceId', 'positionOrder'])
    .groupby('raceId')
    .agg(
        winner=('driver_name', lambda s: s.iloc[0] if len(s) > 0 else None),
        runner_up=('driver_name', lambda s: s.iloc[1] if len(s) > 1 else None),
        third_place=('driver_name', lambda s: s.iloc[2] if len(s) > 2 else None),
        podium_list=('driver_name', lambda s: ', '.join(s)),
    )
    .reset_index()
)

pit_totals = (
    analysis_pit.groupby('raceId')
    .agg(total_pit_stops=('stop', 'count'))
    .reset_index()
)
driver_counts = (
    race_scope.groupby('raceId')['driverId'].nunique()
    .reset_index(name='classified_drivers')
)
pit_summary = pit_totals.merge(driver_counts, on='raceId', how='right').fillna({'total_pit_stops': 0})
pit_summary['total_pit_stops'] = pit_summary['total_pit_stops'].astype(int)
pit_summary['avg_stops_per_driver'] = (
    pit_summary['total_pit_stops'] / pit_summary['classified_drivers']
).round(2)

strategy_summary = (
    strategy_scope.groupby('raceId')
    .agg(
        total_driver_strategies=('driverId', 'size'),
        unique_strategy_count=('strategy_unique', 'sum'),
    )
    .reset_index()
)
strategy_summary['unique_strategy_count'] = strategy_summary['unique_strategy_count'].fillna(0).astype(int)
strategy_summary['unique_strategy_share'] = strategy_summary.apply(
    lambda row: round(row['unique_strategy_count'] / row['total_driver_strategies'], 3)
    if row['total_driver_strategies'] else 0,
    axis=1,
)

position_df = race_scope.copy()
position_df['grid_num'] = pd.to_numeric(position_df['grid'], errors='coerce')
position_df.loc[position_df['grid_num'] <= 0, 'grid_num'] = float('nan')
position_df['positionOrder'] = pd.to_numeric(position_df['positionOrder'], errors='coerce')
position_df['position_gain'] = position_df['grid_num'] - position_df['positionOrder']

position_stats = []
for race_id, group in position_df.groupby('raceId'):
    top10 = group.sort_values('positionOrder').head(10)
    avg_gain_top10 = top10['position_gain'].dropna().mean()
    valid = group.dropna(subset=['position_gain'])
    if not valid.empty:
        top_idx = valid['position_gain'].idxmax()
        top_mover = valid.loc[top_idx, 'driver_name']
        top_gain = valid.loc[top_idx, 'position_gain']
    else:
        top_mover = None
        top_gain = None
    position_stats.append(
        {
            'raceId': race_id,
            'avg_position_gain_top10': round(avg_gain_top10, 2) if pd.notna(avg_gain_top10) else None,
            'top_mover': top_mover,
            'top_mover_gain': float(top_gain) if pd.notna(top_gain) else None,
        }
    )

position_summary = pd.DataFrame(position_stats)

regret_summary = (
    regret_scope.groupby('raceId')['regret_vs_winner_ms']
    .agg(max_regret_ms='max', median_regret_ms='median')
    .reset_index()
)
regret_summary['max_regret_s'] = (regret_summary['max_regret_ms'] / 1000).round(3)
regret_summary['median_regret_s'] = (regret_summary['median_regret_ms'] / 1000).round(3)

race_scope['is_classified_finish'] = (
    race_scope['status_label'].str.contains('Finished', case=False, na=False)
    | race_scope['status_label'].str.startswith('+', na=False)
)
dnf_summary = (
    race_scope.groupby('raceId')
    .agg(
        total_classified=('driverId', 'size'),
        classified_finishers=('is_classified_finish', 'sum'),
    )
    .reset_index()
)
dnf_summary['dnf_count'] = dnf_summary['total_classified'] - dnf_summary['classified_finishers']

race_info = races[races['season'].isin(target_summary_seasons)][['raceId', 'season', 'race_name', 'date', 'round']].copy()

race_features = reduce(
    lambda left, right: left.merge(right, on='raceId', how='left'),
    [
        race_info,
        podium_df,
        pit_summary,
        strategy_summary,
        position_summary,
        regret_summary[['raceId', 'max_regret_s', 'median_regret_s']],
        dnf_summary[['raceId', 'dnf_count', 'total_classified']],
    ],
)
race_features['unique_strategy_count'] = race_features['unique_strategy_count'].fillna(0).astype(int)
race_features['total_driver_strategies'] = race_features['total_driver_strategies'].fillna(0).astype(int)
race_features['total_pit_stops'] = race_features['total_pit_stops'].fillna(0).astype(int)
race_features['avg_stops_per_driver'] = race_features['avg_stops_per_driver'].fillna(0).round(2)
race_features['dnf_count'] = race_features['dnf_count'].fillna(0).astype(int)
race_features['top_mover_gain'] = race_features['top_mover_gain'].round(1)
race_features = race_features.sort_values(['season', 'round']).reset_index(drop=True)

print(race_features[['season', 'race_name', 'unique_strategy_share', 'avg_stops_per_driver']].head())
print(race_features[['season', 'race_name', 'dnf_count', 'max_regret_s']].tail())


KeyError: "['race_name'] not in index"

In [ ]:
def build_race_summary(row):
    sentences = []
    podium_bits = [bit for bit in [row.get('winner'), row.get('runner_up'), row.get('third_place')] if isinstance(bit, str)]
    if podium_bits:
        lead = f"{row['race_name']} {row['season']} saw {podium_bits[0]} take the win"
        if len(podium_bits) > 1:
            lead += f", ahead of {podium_bits[1]}"
        if len(podium_bits) > 2:
            lead += f" and {podium_bits[2]}"
        lead += '.'
        sentences.append(lead)

    if pd.notna(row.get('unique_strategy_share')) and row['unique_strategy_share'] > 0:
        share_pct = int(row['unique_strategy_share'] * 100)
        if share_pct >= 35:
            sentences.append(f"Strategy roulette defined the race, with {share_pct}% of drivers running one-of-one pit plans.")
        elif share_pct >= 20:
            sentences.append(f"Creative pit timing paid off: {share_pct}% of the grid tried unique stop patterns.")
    if pd.notna(row.get('avg_stops_per_driver')) and row['avg_stops_per_driver'] >= 2.5:
        sentences.append(f"High degradation forced an average of {row['avg_stops_per_driver']:.1f} stops per driver.")
    if pd.notna(row.get('top_mover_gain')) and row['top_mover_gain'] >= 5 and isinstance(row.get('top_mover'), str):
        sentences.append(f"{row['top_mover']} was the top mover, gaining {int(row['top_mover_gain'])} grid spots.")
    if pd.notna(row.get('dnf_count')) and row['dnf_count'] >= 5:
        sentences.append(f"Attrition was heavy with {row['dnf_count']} retirements.")
    if pd.notna(row.get('max_regret_s')) and row['max_regret_s'] >= 30:
        sentences.append(f"The biggest strategy miss cost about {int(row['max_regret_s'])} seconds.")
    if not sentences:
        sentences.append('A steady race delivered a straightforward finish for the front-runners.')
    return ' '.join(sentences)

race_features['summary_text'] = race_features.apply(build_race_summary, axis=1)
race_features['youtube_highlights_search'] = race_features.apply(
    lambda row: f"https://www.youtube.com/results?search_query={quote_plus(str(row['season']) + ' ' + row['race_name'] + ' F1 highlights')}",
    axis=1,
)

race_summaries = race_features[[
    'season', 'round', 'race_name', 'summary_text', 'youtube_highlights_search',
    'unique_strategy_share', 'avg_stops_per_driver', 'dnf_count', 'max_regret_s', 'top_mover', 'top_mover_gain',
]].copy()

race_summaries.head(5)


In [ ]:
race_summaries[race_summaries['season'] == 2024].head(3)


## Next Steps

- Validate lap-time derived stint segments against pit windows for a handful of sample races.
- Enrich the regret proxy by incorporating pit loss estimates and compound choices per stint.
- Persist cleaned tables to Parquet/CSV to feed the Next.js app or Supabase loading scripts.
